# 1. 抓取特定Fanpage前10頁POST文 並將其存入到 CSV

## 以兩個粉絲頁為例
### https://www.facebook.com/ETtodaySTAR/ ; https://www.facebook.com/imDataMan/

## 在執行程式前 需先至FB Gragh API申請 
## 暫時權杖(Access Token)
### https://developers.facebook.com/tools/explorer
## OR
## 長時權杖(Access Token)
### https://developers.facebook.com/tools/accesstoken/
## 完整說明請參考https://developers.facebook.com/docs/graph-api/

In [ ]:
import requests
import pandas as pd 
from dateutil.parser import parse

In [ ]:
MyToken = "EAACEdEose0cBAAYIx1ore2pRPN00FisZAVYdZCALZCFj9Q0vBHSJ1NOWzdvMkslPlIZCXnQOZADxcZC1ovZBXanP5BztsWdj0CiUXrAISn3XXZBdKSnuZCwuis9RTaM61UZCTGi5fva3ZBzpsOwjR1nf1p8iQX7Drwy3qNDs9g4eRBCnanofscwgrzi392QTwjHZCB0ZD"

In [ ]:
def getUserInputForURLs():
    urllist = []
    userinput = ''
    while True:
        print("請輸入facebook's fanpage\n")
        userinput = input("輸入Q or q 結束")
        if userinput == 'q' or userinput == 'Q':
            break
        else:
            urllist.append(userinput+'?fref=ts')
    return urllist

In [ ]:
def getIDs(urllist, MyToken):
    fanpages = {}
    for ele in urllist:
        res = requests.get('https://graph.facebook.com/v2.8/{}&access_token={}'.format(ele, MyToken))
        print(res.json())
        if 'og_object' in res.json():
            print(res.json()['og_object']['id'])
            resID = res.json()['og_object']['id']
            fanpages[resID] =  res.json()['id']

        else:
            res.json()
            resID = res.json()['id']
            fanpages[resID] =  res.json()['name']
            print(res.json()['id'])
    return fanpages

In [ ]:
def getPosts(fanpage, MyToken):
    #建立一個空的list          
    information_list = []
    #使用for迴圈依序讀取粉絲頁的資訊，並使用format將id與token傳入{}裡
    for ele in fanpages:
        res = requests.get('https://graph.facebook.com/v2.8/{}/posts?limit=100&access_token={}'.format(ele, MyToken))
        count = 0 
        while 'paging' in res.json() and count < 10:
            count += 1
            for information in res.json()['data']:
                if 'message' in information:
                    information_list.append([fanpage[ele], information['message'], parse(information['created_time']).date()])
            try:
                res = requests.get(res.json()['paging']['next'])
            except:
                pass
    return information_list

In [ ]:
#main ebtry
#https://www.facebook.com/ETtodaySTAR/ ; https://www.facebook.com/imDataMan/
print("Facebook爬蟲主程式開始")
urllist = getUserInputForURLs()
fanpages = getIDs(urllist, MyToken)
information_list = getPosts(fanpages,MyToken)
information_df = pd.DataFrame(information_list, columns=['粉絲專頁', '發文內容', '發文時間']) 
information_df.to_csv('Facebook爬文內容_1.csv', index=False) 

In [ ]:
information_df

In [ ]:
urllist

# 2. 搜尋符合主題的Fanpage並將抓取前2頁POST文 並將其存入到 CSV

In [21]:
import requests
import pandas as pd 
from dateutil.parser import parse
MyToken = "EAACEdEose0cBAAYIx1ore2pRPN00FisZAVYdZCALZCFj9Q0vBHSJ1NOWzdvMkslPlIZCXnQOZADxcZC1ovZBXanP5BztsWdj0CiUXrAISn3XXZBdKSnuZCwuis9RTaM61UZCTGi5fva3ZBzpsOwjR1nf1p8iQX7Drwy3qNDs9g4eRBCnanofscwgrzi392QTwjHZCB0ZD"

In [22]:
def getUserInputForSearch():
    topic_list = []
    userinput = ''
    while True:
        print("請輸入要搜尋facebook's fanpage\n")
        userinput = input("輸入Q or q 結束")
        if userinput == 'q' or userinput == 'Q':
            break
        else:
            topic_list.append(userinput)
    return topic_list

In [23]:
def getIDs(topic_list, MyToken):
    fanpages = {}
    for ele in topic_list:
        #search?q=python&type=page
        res = requests.get('https://graph.facebook.com/v2.8/search?q={}&type=page&access_token={}'.format(ele, MyToken))
        #print(res.json())
        for i in res.json()['data']:
            resID = i['id']
            fanpages[resID] = i['name']

    return fanpages

In [27]:
def getPosts(fanpage, MyToken):
    import time
    start = time.time()
    #建立一個空的list          
    information_list = []
    #使用for迴圈依序讀取粉絲頁的資訊，並使用format將id與token傳入{}裡
    for ele in fanpages:
        res = requests.get('https://graph.facebook.com/v2.8/{}/posts?limit=5&access_token={}'.format(ele, MyToken))
        count = 0 
        while 'paging' in res.json() and count < 2:
            count += 1
            for information in res.json()['data']:
                if 'message' in information:
                    information_list.append([fanpage[ele], information['message'], parse(information['created_time']).date()])
            try:
                res = requests.get(res.json()['paging']['next'])
            except:
                pass
    done = time.time()
    elapsed = done - start
    print("總共花費 %d secs!" %(elapsed)) 
    return information_list

In [31]:
topic_list = getUserInputForSearch()
print("總共有 %d 主題!" % (len(topic_list))) 
fanpages = getIDs(topic_list, MyToken)
print("相關共有 %d Fanpages!" % (len(fanpages))) 
information_list = getPosts(fanpages, MyToken)
print("總共抓了 %d Post文!" %(len(information_list))) 
information_df = pd.DataFrame(information_list, columns=['粉絲專頁', '發文內容', '發文時間']) 
information_df.to_csv('Facebook搜尋post內容_1.csv', index=False) 

請輸入要搜尋facebook's fanpage

輸入Q or q 結束茹絲葵
請輸入要搜尋facebook's fanpage

輸入Q or q 結束q
總共有 1 主題!
相關共有 25 Fanpages!
總共花費 31 secs!
總共抓了 19 Post文!


In [32]:
information_df

,粉絲專頁,發文內容,發文時間
0,Ruth's Chris Steak House茹絲葵經典牛排館台灣,茹絲葵母親節套餐(Mother's Day Menu)\n\n讓我們為這個特別日子裡的美好相...,2017-05-08
1,Ruth's Chris Steak House茹絲葵經典牛排館台灣,茹絲葵情人節套餐(Valentine’s Menu 2017)\n堅持選用只佔市場不到2%的...,2017-02-10
2,Ruth's Chris Steak House茹絲葵經典牛排館台灣,提前和大家拜個年，祝願各位新春佳節愉快！\n\n茹絲葵新年時間營業時間公告如下：\n1/27...,2017-01-19
3,Ruth's Chris Steak House茹絲葵經典牛排館台灣,歲末年終之際，茹絲葵為您推薦俄國沙皇指定的\n黃金佳釀「水晶香檳」(Cristal Cham...,2016-12-13
4,Ruth's Chris Steak House茹絲葵經典牛排館台灣,歲末年終之際，最適合用美食佳釀犒賞自己！\n\n***茹絲葵推出2016節慶套餐Holida...,2016-11-25
5,Ruth's Chris Steak House茹絲葵經典牛排館台灣,挑選多汁的蘑菇鑲入半顆拳頭大小的蟹肉\n蟹餡綿密、鮮甜，搭配羅瑪諾起司和西芹等蔬菜完美調味，...,2016-11-04
6,Ruth's Chris Steak House茹絲葵經典牛排館台灣,以兩種純度比例不同的巧克力結合Espresso\n不加入任何蛋糕體和麵粉做成蛋糕\n\n香濃...,2016-10-24
7,Ruth's Chris Steak House茹絲葵經典牛排館台灣,免去與蟹殼纏鬥，以拳頭般大小扎實的蟹餅\n優雅呈現\n\n極鮮的蟹肉，佐以檸檬奶油香\n在保...,2016-09-30
8,Ruth's Chris Steak House茹絲葵經典牛排館台灣,頂級牛排當前，我們從不失手！\nWe don’t fool around when it c...,2016-08-28
9,炤茹絲葵,"🍃鹹鹹市集🍃\n結束了兩天的市集,收穫滿滿的\n隔壁的香港弟弟妹妹很乖的陪媽媽來,妹妹也喜歡...",2016-04-25


412